<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/main/18_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install eciespy eth_keys cryptos qrcode
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git
! cp Colegio_Invisible/modules/colegio_tools.py colegio_tools.py

In [ ]:
from colegio_tools import *

# Now We Have Colegio Tools

## Load Real Keys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp drive/MyDrive/cinv/llaves.zip llaves.zip 
! unzip llaves.zip

In [ ]:
my_qr=IPython.display.Image('llaves/mi_addr.png')
my_privKey=import_privKey('llaves/mi_prv.enc','')
my_pub=import_pubKey('llaves/mi_pub.bin')
my_addr=import_addr('llaves/mi_addr.bin')

display(my_qr)
print('*'*50)
print('PUBLIC:',my_pub)
print('ADDR:',my_addr)

import cryptos
doge=cryptos.Doge()

doge_unspents=doge.unspent(my_addr) 
doge_unspent_total=sum([unspent['value'] for unspent in doge_unspents])

print('*'*50)
print('BALANCE:\t',doge_unspent_total/10**8)
print('NUM_UTXO:\t',len(doge_unspents))
print('*'*50)

# Accessing Encypted Files From Blockchain

We will be accessing an encrypted color image file woven from transaction:

```d0209a0f85872d6826c58bc23fab37c8b21feb22c15a5a6469f45358fb78ba41```

Remeber the header data contains the following:
```text
b'\xc1\xdd\x00\x01\x0e\x03\x01'+length_bytes+width_bytes+bitnum_bytes+recip_bytes... + some_text_bytes
```

In [ ]:
Txn='d0209a0f85872d6826c58bc23fab37c8b21feb22c15a5a6469f45358fb78ba41'

(head_d02,bytes_d02)=read_cadenas(Txn)

num_keys=int(head_d02[24:26])
zipkeys=[bytes_d02[i*64:i*64+64] for i in range(num_keys) ]
zipdata=bytes_d02[num_keys*64:]
print('HEADER:',head_d02)

txn_pub_hex=get_txn_pub(Txn)
print('AUTHOR:',txn_pub_hex)
print('NUM_RECIP:',num_keys)

In [ ]:
N=0

txn_pubKey=eth_keys.keys.PublicKey(bytes.fromhex(txn_pub_hex))
my_shared_key=shared_key(my_privKey,txn_pubKey)
ses_kee=ecies.aes_decrypt(my_shared_key,zipkeys[N])
data_do2=ecies.aes_decrypt(ses_kee,zipdata)
array_do2=read_image_data(head_d02,data_do2)

Image.fromarray(array_do2)

In [ ]:

(d68_head,d68_array)=array_dec_from_txn('d68175766b70f7163aec93e5a4e81480a6c6dd51d0577319d6f3392e8725f53b',my_privKey,0)
Image.fromarray(d68_array)